<a href="https://colab.research.google.com/github/alimomennasab/CS4630/blob/main/CS4630.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
#
import numpy as np
from google.colab import drive
drive.mount('/content/drive')